In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [4]:
data=pd.read_csv("광주아파트매매데이터.CSV", encoding= "EUC-KR")

In [5]:
data.head(5)

,행정명,번지,본번,부번,단지명,단지명길이,전용면적(㎡),계약년월,계약년,계약월,계약일,거래금액(만원),층,건축년도,건물나이,도로명,해제사유발생일,거래유형,중개사소재지
0,광산구 도산동,1138-2,1138,2,대덕1,3,59.3100,202303,2023,3,18,9800,4,1990,33,송도로 143,NaN,중개거래,광주 광산구
1,광산구 도산동,790,790,0,대주피오레1차,7,84.9800,202302,2023,2,7,21000,2,2000,23,송도로182번길 55,NaN,중개거래,광주 광산구
2,광산구 도산동,790,790,0,대주피오레1차,7,84.9800,202305,2023,5,31,22350,5,2000,23,송도로182번길 55,NaN,중개거래,광주 광산구
3,광산구 도산동,820,820,0,대주피오레2차,7,84.7575,202305,2023,5,9,22400,5,2001,22,송도로182번길 16-3,NaN,직거래,-
4,광산구 도산동,820,820,0,대주피오레2차,7,84.7575,202305,2023,5,13,22000,13,2001,22,송도로182번길 16-3,NaN,중개거래,광주 광산구


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425079 entries, 0 to 425078
Data columns (total 19 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   행정명       425079 non-null  object 
 1   번지        425079 non-null  object 
 2   본번        425079 non-null  int64  
 3   부번        425079 non-null  int64  
 4   단지명       425079 non-null  object 
 5   단지명길이     425079 non-null  int64  
 6   전용면적(㎡)   425079 non-null  float64
 7   계약년월      425079 non-null  int64  
 8   계약년       425079 non-null  int64  
 9   계약월       425079 non-null  int64  
 10  계약일       425079 non-null  int64  
 11  거래금액(만원)  425079 non-null  int64  
 12  층         425079 non-null  int64  
 13  건축년도      425079 non-null  int64  
 14  건물나이      425079 non-null  int64  
 15  도로명       425079 non-null  object 
 16  해제사유발생일   3417 non-null    float64
 17  거래유형      75324 non-null   object 
 18  중개사소재지    75324 non-null   object 
dtypes: float64(2), int64(11), object(6)
memory u

In [7]:
data=data.drop(['번지','본번','부번','단지명','도로명','해제사유발생일','거래유형','중개사소재지'], axis=1)

In [8]:
data

,행정명,단지명길이,전용면적(㎡),계약년월,계약년,계약월,계약일,거래금액(만원),층,건축년도,건물나이
0,광산구 도산동,3,59.3100,202303,2023,3,18,9800,4,1990,33
1,광산구 도산동,7,84.9800,202302,2023,2,7,21000,2,2000,23
2,광산구 도산동,7,84.9800,202305,2023,5,31,22350,5,2000,23
3,광산구 도산동,7,84.7575,202305,2023,5,9,22400,5,2001,22
4,광산구 도산동,7,84.7575,202305,2023,5,13,22000,13,2001,22
...,...,...,...,...,...,...,...,...,...,...,...
425074,서구 화정동,4,115.1850,200611,2006,11,10,19000,17,1996,10
425075,서구 화정동,4,84.8300,200611,2006,11,27,12000,6,1996,10
425076,서구 화정동,4,59.8300,200611,2006,11,30,7700,2,1996,10
425077,서구 화정동,4,84.8400,200612,2006,12,6,12170,16,1996,10


In [9]:
data['행정명'].unique()

array([' 광산구 도산동', ' 광산구 도천동', ' 광산구 비아동', ' 광산구 산월동', ' 광산구 산정동',
       ' 광산구 선암동', ' 광산구 소촌동', ' 광산구 송정동', ' 광산구 수완동', ' 광산구 신가동',
       ' 광산구 신창동', ' 광산구 신촌동', ' 광산구 쌍암동', ' 광산구 우산동', ' 광산구 운남동',
       ' 광산구 운수동', ' 광산구 월계동', ' 광산구 월곡동', ' 광산구 장덕동', ' 광산구 하남동',
       ' 광산구 흑석동', ' 남구 노대동', ' 남구 방림동', ' 남구 백운동', ' 남구 봉선동', ' 남구 사동',
       ' 남구 서동', ' 남구 송하동', ' 남구 압촌동', ' 남구 양림동', ' 남구 월산동', ' 남구 임암동',
       ' 남구 주월동', '남구 주월동', ' 남구 지석동', ' 남구 진월동', ' 남구 행암동', ' 동구 계림동',
       ' 동구 금남로2가', ' 동구 금남로3가', ' 동구 금남로5가', ' 동구 대인동', ' 동구 동명동',
       ' 동구 산수동', ' 동구 소태동', ' 동구 수기동', ' 동구 용산동', ' 동구 운림동', ' 동구 월남동',
       ' 동구 지산동', ' 동구 충장로4가', ' 동구 학동', ' 북구 각화동', ' 북구 동림동', ' 북구 두암동',
       ' 북구 매곡동', ' 북구 문흥동', ' 북구 본촌동', ' 북구 북동', ' 북구 삼각동', ' 북구 신안동',
       ' 북구 신용동', ' 북구 양산동', ' 북구 연제동', ' 북구 오치동', ' 북구 용두동', ' 북구 용봉동',
       ' 북구 우산동', ' 북구 운암동', ' 북구 유동', ' 북구 일곡동', ' 북구 임동', ' 북구 중흥동',
       ' 북구 풍향동', ' 서구 광천동', ' 서구 금호동', ' 서구 내방동', ' 서구 농성동', ' 서구 동천동',
       ' 서구

In [10]:
len(data['행정명'].unique())

87

feature_importance = rf_model.feature_importances_

    # 특성 중요도 시각화
    plt.figure(figsize=(10, 6))
    plt.bar(X_train.columns, feature_importance)
    plt.xticks(rotation=45)
    plt.ylabel("Feature Importance")
    plt.title("Feature Importance of Variables (RandomForest)")
    plt.show()

In [11]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 계약날짜를 datetime 형식으로 변환
data['계약날짜'] = pd.to_datetime(data['계약년월'].astype(str) + data['계약일'].astype(str), format='%Y%m%d')
# 입력 변수(X)와 타겟 변수(y) 설정
X = data[['단지명길이', '전용면적(㎡)', '층', '건물나이']]
y = data['거래금액(만원)']
#행정명 변수를 원핫인코딩하여 더미 변수로 변환
onehot_encoded = pd.get_dummies(data['행정명'])

# X에 원핫인코딩된 더미 변수를 추가하여 새로운 X 데이터프레임 생성
X = pd.concat([X, onehot_encoded], axis=1)

# 앙상블과 시계열 모델 결합하여 집값 예측 함수
def combined_prediction(timeseries_data, features, target, weight_arima=0.5, weight_rf=0.5):
    # 시계열 모델 학습
    model = ARIMA(timeseries_data, order=(1, 1, 1))  # ARIMA 모델의 파라미터는 적절히 설정해야 합니다.
    model_fit = model.fit()
    arima_forecast = model_fit.forecast(steps=len(X_test))  # 테스트 데이터 크기만큼 미래 값을 예측

    # 앙상블 모델 학습
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # 랜덤 포레스트 모델의 파라미터는 적절히 설정해야 합니다.
    rf_model.fit(features, target)
    rf_forecast = rf_model.predict(X_test)
       
    # 시계열 모델과 앙상블 모델의 예측값을 결합하여 최종 예측값 생성
    final_prediction = (arima_forecast * weight_arima + rf_forecast * weight_rf) / (weight_arima + weight_rf)
    return final_prediction

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
final_prediction = combined_prediction(data.set_index('계약날짜')['거래금액(만원)'], X_train, y_train, weight_arima=0.5, weight_rf=0.5)
print("Final Prediction:", final_prediction)



# 모델 평가
mse = mean_squared_error(y_test, final_prediction)
r2 = r2_score(y_test, final_prediction)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. for

Final Prediction: 425079    13192.388042
425080    10489.801852
425081    18484.957766
425082    14436.060860
425083    13996.732203
              ...     
510090    21409.951653
510091    11907.004090
510092    17196.076653
510093    29479.890343
510094    15069.007683
Name: predicted_mean, Length: 85016, dtype: float64
Mean Squared Error: 43337014.539724074
R-squared: 0.7203768300538842


In [12]:
print("Final Prediction:", final_prediction.mean())

Final Prediction: 15718.27307093566


In [13]:
X_predict = [[12, 84, 14, 2020, '2024-4-1']]

In [25]:
X_predict_data = pd.DataFrame([[12, 84, 10, 2020]], columns=['단지명길이', '전용면적(㎡)', '층', '건물나이'])
X_predict_onehot = pd.get_dummies(pd.Series(['광주광역시 동구 운림동']), prefix='시군구')
X_predict_data = pd.concat([X_predict_data, X_predict_onehot], axis=1)



SyntaxError: positional argument follows keyword argument (1627391200.py, line 6)

In [20]:
 predicted_price.mean()
    

15718.27307093566

In [26]:
# 계약날짜를 datetime 형식으로 변환
data['계약날짜'] = pd.to_datetime(data['계약년월'].astype(str) + data['계약일'].astype(str), format='%Y%m%d')

# 입력 변수(X)와 타겟 변수(y) 설정
X = data[['단지명길이', '전용면적(㎡)', '층', '건물나이']]
y = data['거래금액(만원)']

# 행정명 변수를 원핫인코딩하여 더미 변수로 변환
onehot_encoded = pd.get_dummies(data['행정명'])
X = pd.concat([X, onehot_encoded], axis=1)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 시계열 모델 학습
model = ARIMA(data.set_index('계약날짜')['거래금액(만원)'], order=(1, 1, 1))
model_fit = model.fit()

# 앙상블 모델 학습
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 시계열 모델의 예측값을 생성
arima_forecast = model_fit.forecast(steps=len(X_test))

# 앙상블 모델의 예측값을 생성
rf_forecast = rf_model.predict(X_test)

# 시계열 모델과 앙상블 모델의 예측값을 결합하여 최종 예측값 생성
final_prediction = (arima_forecast * 0.5 + rf_forecast * 0.5)  # 예측값 결합 방식은 가중치를 조절하여 조정 가능

print("Final Prediction:", final_prediction)

# 모델 평가
mse = mean_squared_error(y_test, final_prediction)
r2 = r2_score(y_test, final_prediction)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. for

Final Prediction: 425079    13192.388042
425080    10489.801852
425081    18484.957766
425082    14436.060860
425083    13996.732203
              ...     
510090    21409.951653
510091    11907.004090
510092    17196.076653
510093    29479.890343
510094    15069.007683
Name: predicted_mean, Length: 85016, dtype: float64
Mean Squared Error: 43337014.539724074
R-squared: 0.7203768300538842


In [31]:
X_predict_data = pd.DataFrame([[12, 84, 10, 2020]], columns=['단지명길이', '전용면적(㎡)', '층', '건물나이'])
X_predict_data['행정명'] = '동구 운림동'

onehot_cols = list(onehot_encoded.columns)

# 예측 시에도 동일한 컬럼들로 원핫인코딩 수행
X_predict_onehot = pd.get_dummies(X_predict_data['행정명'], prefix='행정명')
X_predict_data = pd.concat([X_predict_data, X_predict_onehot], axis=1)

# 모델의 예측값을 생성
arima_forecast_predict = model_fit.forecast(steps=len(X_predict_data))
rf_forecast_predict = rf_model.predict(X_predict_data)

# 예측값 결합
prediction_predict = (arima_forecast_predict * 0.5 + rf_forecast_predict * 0.5)

print("Final Prediction:", prediction_predict)

C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 행정명
- 행정명_동구 운림동
Feature names seen at fit time, yet now missing:
-  광산구 도산동
-  광산구 도천동
-  광산구 비아동
-  광산구 산월동
-  광산구 산정동
- ...


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



# 계약날짜를 datetime 형식으로 변환
data['계약날짜'] = pd.to_datetime(data['계약년월'].astype(str) + data['계약일'].astype(str), format='%Y%m%d')

# 입력 변수(X)와 타겟 변수(y) 설정
X = data[['단지명길이', '전용면적(㎡)', '층', '건물나이']]
y = data['거래금액(만원)']

# 행정명 변수를 원핫인코딩하여 더미 변수로 변환
onehot_encoded = pd.get_dummies(data['행정명'])

# X에 원핫인코딩된 더미 변수를 추가하여 새로운 X 데이터프레임 생성
X = pd.concat([X, onehot_encoded], axis=1)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 앙상블과 시계열 모델 결합하여 집값 예측 함수
def combined_prediction(timeseries_data, features, target, weight_arima=0.5, weight_rf=0.5):
    # 시계열 모델 학습
    model = ARIMA(timeseries_data, order=(1, 1, 1))  # ARIMA 모델의 파라미터는 적절히 설정해야 합니다.
    model_fit = model.fit()
    arima_forecast = model_fit.forecast(steps=len(X_test))  # 테스트 데이터 크기만큼 미래 값을 예측

    # 앙상블 모델 학습
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # 랜덤 포레스트 모델의 파라미터는 적절히 설정해야 합니다.
    rf_model.fit(features, target)
    rf_forecast = rf_model.predict(X_test)

    # 시계열 모델과 앙상블 모델의 예측값을 결합하여 최종 예측값 생성
    final_prediction = (arima_forecast * weight_arima + rf_forecast * weight_rf) / (weight_arima + weight_rf)
    return final_prediction

# 모델 학습
final_prediction = combined_prediction(data.set_index('계약날짜')['거래금액(만원)'], X_train, y_train, weight_arima=0.5, weight_rf=0.5)

# 모델 평가
mse = mean_squared_error(y_test, final_prediction)
r2 = r2_score(y_test, final_prediction)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

# X_predict_data를 생성하여 예측값 출력
X_predict_data = pd.DataFrame([[12, 84, 10, 2020]], columns=['단지명길이', '전용면적(㎡)', '층', '건물나이'])
X_predict_onehot = pd.get_dummies(pd.Series(['동구 운림동']), prefix='행정명')
X_predict_data = pd.concat([X_predict_data, X_predict_onehot], axis=1)

# 시계열 모델의 예측값을 생성
model = ARIMA(data.set_index('계약날짜')['거래금액(만원)'], order=(1, 1, 1))  
model_fit = model.fit()
arima_forecast_predict = model_fit.forecast(steps=len(X_predict_data))

# 앙상블 모델의 예측값을 생성
rf_forecast_predict = rf_model.predict(X_predict_data)

# 시계열 모델과 앙상블 모델의 예측값을 결합하여 최종 예측값 생성
prediction_predict = (arima_forecast_predict * 0.5 + rf_forecast_predict * 0.5)

print("Final Prediction for X_predict_data:", prediction_predict)


import matplotlib.pyplot as plt

# 랜덤 포레스트와 ARIMA의 예측 결과 비교를 위해 시계열 인덱스 생성
date_range = pd.date_range(start=data['계약날짜'].min(), periods=len(final_prediction), freq='D')

# 예측 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(date_range, y_test.values, label='Actual Price', color='blue')
plt.plot(date_range, final_prediction, label='Predicted Price', color='red')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Actual vs. Predicted House Prices')
plt.legend()
plt.grid(True)
plt.show()